In [1]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [9]:
from selenium import webdriver as wd
from bs4 import BeautifulSoup
import time
import requests
import urllib.request
from selenium.webdriver.common.by import By
import csv



fields = ['Rank', 'Name','FloorPrice', 'AllTimeAvg']
with open('all_time_avg.csv', 'w',newline='') as f: 
    write = csv.writer(f) 
    write.writerow(fields) 
    
testlist = [[1, 2, 3, 4], [1,2,3,4]]
with open('all_time_avg.csv', 'a', newline='', encoding='UTF-8') as f: 
    write = csv.writer(f) 
    write.writerows(testlist) 

dr = wd.Chrome('chromedriver')
url = "https://opensea.io/rankings?sortBy=total_volume&chain=klaytn&category=collectibles"
dr.get(url)
time.sleep(3) 


reslist=[]
linklist=[]
while True:
    while True: 
        before_location = dr.execute_script("return window.pageYOffset") 
        time.sleep(3)
        elements = dr.find_elements(By.XPATH, value='//a[@class="styles__StyledLink-sc-l6elh8-0 ekTmzq Blockreact__Block-sc-1xf18x6-0 Flexreact__Flex-sc-1twd32i-0 Itemreact__ItemBase-sc-1idymv7-0 styles__FullRowContainer-sc-12irlp3-0 bnxsDk jYqxGr dCVDRE lcvzZN fresnel-greaterThanOrEqual-xl"]') 
        for eachelem in elements:
            link = eachelem.get_attribute("href")
            rankname = eachelem.find_elements(By.XPATH, value='*')[0]
            try:
                rank = rankname.find_element(By.XPATH, value='.//div[@class="Overflowreact__OverflowContainer-sc-7qr9y8-0 jPSCbX"]').text
            except:
                continue
            try:
                name = rankname.find_element(By.XPATH, value='.//div[@class="Overflowreact__OverflowContainer-sc-7qr9y8-0 jPSCbX Ranking--collection-name-overflow"]').text
            except:
                continue
            if not((rank == '') or (name == '')) :
                linklist.append((int(rank), name, link))
        dr.execute_script("window.scrollTo(0,{})".format(before_location + 2500))
        after_location = dr.execute_script("return window.pageYOffset")
        if before_location == after_location: 
            break
    prevNextButtons = dr.find_elements(By.XPATH, value='//div[@class="Blockreact__Block-sc-1xf18x6-0 Flexreact__Flex-sc-1twd32i-0 krGFYb jYqxGr"]/*')
    if prevNextButtons[1].get_property('disabled'):
        break
    else:
        prevNextButtons[1].click()


linklist = list(set(linklist))
linklist.sort(key = lambda x : x[0])
print(len(linklist))

for collection in linklist:
    dr.get(collection[2]+"?tab=activity")
    time.sleep(4)
    infobox = dr.find_element(By.XPATH, value='//div[@class="Blockreact__Block-sc-1xf18x6-0 Flexreact__Flex-sc-1twd32i-0 InfoContainerreact__InfoContainer-sc-15x3z7c-0 CollectionStatsBarreact__Container-sc-8gdi9o-0 elqhCm jYqxGr fprnFG cWlCaZ"]')
    floorbox = infobox.find_elements(By.XPATH, value='*')[2]
    floor = floorbox.find_element(By.XPATH, value='.//div[@class="Overflowreact__OverflowContainer-sc-7qr9y8-0 jPSCbX"]').text

    try:
        button = dr.find_element(By.XPATH, value = '//div[@class="Inputreact__StyledContainer-sc-3dr67n-0 iAeYiQ Selectreact__SelectInput-sc-1shssly-0 cJLIjY"]')
    except:
        print((collection[0], collection[1], floor, "no activity"))
        reslist.append([collection[0], collection[1], floor, "no activity"])
        continue
    try:
        button.click()
    except:
        dr.get(collection[2]+"?tab=activity")
        time.sleep(4)
        infobox = dr.find_element(By.XPATH, value='//div[@class="Blockreact__Block-sc-1xf18x6-0 Flexreact__Flex-sc-1twd32i-0 InfoContainerreact__InfoContainer-sc-15x3z7c-0 CollectionStatsBarreact__Container-sc-8gdi9o-0 elqhCm jYqxGr fprnFG cWlCaZ"]')
        floorbox = infobox.find_elements(By.XPATH, value='*')[2]
        floor = floorbox.find_element(By.XPATH, value='.//div[@class="Overflowreact__OverflowContainer-sc-7qr9y8-0 jPSCbX"]').text
        button = dr.find_element(By.XPATH, value = '//div[@class="Inputreact__StyledContainer-sc-3dr67n-0 iAeYiQ Selectreact__SelectInput-sc-1shssly-0 cJLIjY"]')
        button.click()
    time.sleep(2)
    divelem = dr.find_elements(By.XPATH, value='.//li[@class="elements__StyledListItem-sc-197zmwo-0 QbTKh"]')
    divelem[6].click()
    time.sleep(3)
    
    try:
        alltimeavg = dr.find_element(By.XPATH, value='//div[@class="PriceHistoryStats--value"]').text
    except:
        print((collection[0], collection[1], floor, "no All Time Avg"))
        reslist.append([collection[0], collection[1], floor, "no All Time Avg"])
        continue
    print((collection[0], collection[1], floor, alltimeavg[1:]))
    reslist.append([collection[0], collection[1], floor, alltimeavg[1:]])
print(len(reslist))

with open('alltimeavg.csv', 'a', newline='', encoding='UTF-8') as f: 
    write = csv.writer(f) 
    write.writerows(reslist) 



617
(1, 'THE META KONGZ', '15740', '1.3351')
(2, 'Meta Toy DragonZ', '2900', '0.7095')
(3, 'G.rilla OFFICIAL', '1250', '0.8181')
(4, 'DSC | DIGITAL IDENTITY', '350', '0.2891')
(5, 'SheepFarm', '1', '0.0379')
(6, 'Meta Adventure, frozen', '5', '0.2534')
(7, 'METAMON PROJECT', '104', '0.1113')
(8, 'ElectroPixelKongz', '92', '0.0767')
(9, '[Treasures Club] MASTERS', '160', '0.0797')
(10, 'Drawshop Kingdom Reverse', '15', '0.0877')
(11, 'META RABBITS OFFICIAL V1', '165', '0.1118')
(12, 'KLAY APE CLUB', '330', '0.0636')
(13, 'Re-Be World Land', '340', '0.2647')
(14, 'KlayLand', '60', '0.0888')
(15, 'My Fat Babiz', '125', '0.0891')
(16, 'AnimalsPunks V2', '17.7', '0.0303')
(17, 'Kenome', '200', '0.1644')
(18, 'Cat Town Punks', '9', '0.0417')
(19, 'Joseon Dynasty', '89', '0.2158')
(20, 'CyberTHUG Genesis Club', '179', '0.1293')
(21, 'ALAP : THE LOST PIONEERS', '917', '0.2864')
(22, 'PIXEL GROUND REVEAL', '76.49', '0.0665')
(23, 'KLAYRock', '4999', '2.36')
(24, 'METABUMZ-OFFICIAL', '288', '0.1

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=99.0.4844.82)
Stacktrace:
Backtrace:
	Ordinal0 [0x00C79943+2595139]
	Ordinal0 [0x00C0C9F1+2148849]
	Ordinal0 [0x00B043F0+1065968]
	Ordinal0 [0x00B31F17+1253143]
	Ordinal0 [0x00B27AB3+1211059]
	Ordinal0 [0x00B4B9EC+1358316]
	Ordinal0 [0x00B27474+1209460]
	Ordinal0 [0x00B4BC04+1358852]
	Ordinal0 [0x00B5BAF2+1424114]
	Ordinal0 [0x00B4B806+1357830]
	Ordinal0 [0x00B26086+1204358]
	Ordinal0 [0x00B26F96+1208214]
	GetHandleVerifier [0x00E1B232+1658114]
	GetHandleVerifier [0x00ED312C+2411516]
	GetHandleVerifier [0x00D0F261+560433]
	GetHandleVerifier [0x00D0E366+556598]
	Ordinal0 [0x00C1286B+2173035]
	Ordinal0 [0x00C175F8+2192888]
	Ordinal0 [0x00C176E5+2193125]
	Ordinal0 [0x00C211FC+2232828]
	BaseThreadInitThunk [0x767E6739+25]
	RtlGetFullPathName_UEx [0x77548E7F+1215]
	RtlGetFullPathName_UEx [0x77548E4D+1165]


In [11]:
print(linklist)

[(1, 'THE META KONGZ', 'https://opensea.io/collection/the-meta-kongz'), (2, 'Meta Toy DragonZ', 'https://opensea.io/collection/mtdz-1'), (3, 'G.rilla OFFICIAL', 'https://opensea.io/collection/g-rilla-official'), (4, 'DSC | DIGITAL IDENTITY', 'https://opensea.io/collection/dogesoundclub-mates'), (5, 'SheepFarm', 'https://opensea.io/collection/sheepfarm'), (6, 'Meta Adventure, frozen', 'https://opensea.io/collection/meta-adventure-frozen'), (7, 'METAMON PROJECT', 'https://opensea.io/collection/metamon-vial'), (8, 'ElectroPixelKongz', 'https://opensea.io/collection/electropixelkongz'), (9, '[Treasures Club] MASTERS', 'https://opensea.io/collection/treasures-club-masters'), (10, 'Drawshop Kingdom Reverse', 'https://opensea.io/collection/drawshop'), (11, 'META RABBITS OFFICIAL V1', 'https://opensea.io/collection/meta-rabbits-official'), (12, 'KLAY APE CLUB', 'https://opensea.io/collection/klay-ape-club'), (13, 'Re-Be World Land', 'https://opensea.io/collection/re-beland'), (14, 'KlayLand', 

In [15]:
print(reslist)

[[1, 'THE META KONGZ', '15740', '1.3351'], [2, 'Meta Toy DragonZ', '2900', '0.7095'], [3, 'G.rilla OFFICIAL', '1250', '0.8181'], [4, 'DSC | DIGITAL IDENTITY', '350', '0.2891'], [5, 'SheepFarm', '1', '0.0379'], [6, 'Meta Adventure, frozen', '5', '0.2534'], [7, 'METAMON PROJECT', '104', '0.1113'], [8, 'ElectroPixelKongz', '92', '0.0767'], [9, '[Treasures Club] MASTERS', '160', '0.0797'], [10, 'Drawshop Kingdom Reverse', '15', '0.0877'], [11, 'META RABBITS OFFICIAL V1', '165', '0.1118'], [12, 'KLAY APE CLUB', '330', '0.0636'], [13, 'Re-Be World Land', '340', '0.2647'], [14, 'KlayLand', '60', '0.0888'], [15, 'My Fat Babiz', '125', '0.0891'], [16, 'AnimalsPunks V2', '17.7', '0.0303'], [17, 'Kenome', '200', '0.1644'], [18, 'Cat Town Punks', '9', '0.0417'], [19, 'Joseon Dynasty', '89', '0.2158'], [20, 'CyberTHUG Genesis Club', '179', '0.1293'], [21, 'ALAP : THE LOST PIONEERS', '917', '0.2864'], [22, 'PIXEL GROUND REVEAL', '76.49', '0.0665'], [23, 'KLAYRock', '4999', '2.36'], [24, 'METABUMZ-OF

In [16]:
dr = wd.Chrome('chromedriver')
for collection in linklist[109:]:
    dr.get(collection[2]+"?tab=activity")
    time.sleep(4)
    infobox = dr.find_element(By.XPATH, value='//div[@class="Blockreact__Block-sc-1xf18x6-0 Flexreact__Flex-sc-1twd32i-0 InfoContainerreact__InfoContainer-sc-15x3z7c-0 CollectionStatsBarreact__Container-sc-8gdi9o-0 elqhCm jYqxGr fprnFG cWlCaZ"]')
    floorbox = infobox.find_elements(By.XPATH, value='*')[2]
    floor = floorbox.find_element(By.XPATH, value='.//div[@class="Overflowreact__OverflowContainer-sc-7qr9y8-0 jPSCbX"]').text

    try:
        button = dr.find_element(By.XPATH, value = '//div[@class="Inputreact__StyledContainer-sc-3dr67n-0 iAeYiQ Selectreact__SelectInput-sc-1shssly-0 cJLIjY"]')
    except:
        print((collection[0], collection[1], floor, "no activity"))
        reslist.append([collection[0], collection[1], floor, "no activity"])
        continue
    try:
        button.click()
    except:
        dr.get(collection[2]+"?tab=activity")
        time.sleep(4)
        infobox = dr.find_element(By.XPATH, value='//div[@class="Blockreact__Block-sc-1xf18x6-0 Flexreact__Flex-sc-1twd32i-0 InfoContainerreact__InfoContainer-sc-15x3z7c-0 CollectionStatsBarreact__Container-sc-8gdi9o-0 elqhCm jYqxGr fprnFG cWlCaZ"]')
        floorbox = infobox.find_elements(By.XPATH, value='*')[2]
        floor = floorbox.find_element(By.XPATH, value='.//div[@class="Overflowreact__OverflowContainer-sc-7qr9y8-0 jPSCbX"]').text
        button = dr.find_element(By.XPATH, value = '//div[@class="Inputreact__StyledContainer-sc-3dr67n-0 iAeYiQ Selectreact__SelectInput-sc-1shssly-0 cJLIjY"]')
        button.click()
    time.sleep(2)
    divelem = dr.find_elements(By.XPATH, value='.//li[@class="elements__StyledListItem-sc-197zmwo-0 QbTKh"]')
    divelem[6].click()
    time.sleep(3)
    
    try:
        alltimeavg = dr.find_element(By.XPATH, value='//div[@class="PriceHistoryStats--value"]').text
    except:
        print((collection[0], collection[1], floor, "no All Time Avg"))
        reslist.append([collection[0], collection[1], floor, "no All Time Avg"])
        continue
    print((collection[0], collection[1], floor, alltimeavg[1:]))
    reslist.append([collection[0], collection[1], floor, alltimeavg[1:]])
print(len(reslist))

with open('alltimeavg.csv', 'a', newline='', encoding='UTF-8') as f: 
    write = csv.writer(f) 
    write.writerows(reslist) 

(110, 'CrayonBoys', '6', '0.005')
(111, 'goth girl !!', '200', '0.0249')
(112, 'LHS ROOT', '270', '0.1808')


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class="Blockreact__Block-sc-1xf18x6-0 Flexreact__Flex-sc-1twd32i-0 InfoContainerreact__InfoContainer-sc-15x3z7c-0 CollectionStatsBarreact__Container-sc-8gdi9o-0 elqhCm jYqxGr fprnFG cWlCaZ"]"}
  (Session info: chrome=99.0.4844.82)
Stacktrace:
Backtrace:
	Ordinal0 [0x00C79943+2595139]
	Ordinal0 [0x00C0C9F1+2148849]
	Ordinal0 [0x00B04528+1066280]
	Ordinal0 [0x00B30FD4+1249236]
	Ordinal0 [0x00B311CB+1249739]
	Ordinal0 [0x00B5D812+1431570]
	Ordinal0 [0x00B4BA34+1358388]
	Ordinal0 [0x00B5BAF2+1424114]
	Ordinal0 [0x00B4B806+1357830]
	Ordinal0 [0x00B26086+1204358]
	Ordinal0 [0x00B26F96+1208214]
	GetHandleVerifier [0x00E1B232+1658114]
	GetHandleVerifier [0x00ED312C+2411516]
	GetHandleVerifier [0x00D0F261+560433]
	GetHandleVerifier [0x00D0E366+556598]
	Ordinal0 [0x00C1286B+2173035]
	Ordinal0 [0x00C175F8+2192888]
	Ordinal0 [0x00C176E5+2193125]
	Ordinal0 [0x00C211FC+2232828]
	BaseThrea

In [18]:
dr = wd.Chrome('chromedriver')
for collection in linklist[112:]:
    dr.get(collection[2]+"?tab=activity")
    time.sleep(4)
    infobox = dr.find_element(By.XPATH, value='//div[@class="Blockreact__Block-sc-1xf18x6-0 Flexreact__Flex-sc-1twd32i-0 InfoContainerreact__InfoContainer-sc-15x3z7c-0 CollectionStatsBarreact__Container-sc-8gdi9o-0 elqhCm jYqxGr fprnFG cWlCaZ"]')
    floorbox = infobox.find_elements(By.XPATH, value='*')[2]
    floor = floorbox.find_element(By.XPATH, value='.//div[@class="Overflowreact__OverflowContainer-sc-7qr9y8-0 jPSCbX"]').text

    try:
        button = dr.find_element(By.XPATH, value = '//div[@class="Inputreact__StyledContainer-sc-3dr67n-0 iAeYiQ Selectreact__SelectInput-sc-1shssly-0 cJLIjY"]')
    except:
        print((collection[0], collection[1], floor, "no activity"))
        reslist.append([collection[0], collection[1], floor, "no activity"])
        continue
    try:
        button.click()
    except:
        dr.get(collection[2]+"?tab=activity")
        time.sleep(4)
        infobox = dr.find_element(By.XPATH, value='//div[@class="Blockreact__Block-sc-1xf18x6-0 Flexreact__Flex-sc-1twd32i-0 InfoContainerreact__InfoContainer-sc-15x3z7c-0 CollectionStatsBarreact__Container-sc-8gdi9o-0 elqhCm jYqxGr fprnFG cWlCaZ"]')
        floorbox = infobox.find_elements(By.XPATH, value='*')[2]
        floor = floorbox.find_element(By.XPATH, value='.//div[@class="Overflowreact__OverflowContainer-sc-7qr9y8-0 jPSCbX"]').text
        button = dr.find_element(By.XPATH, value = '//div[@class="Inputreact__StyledContainer-sc-3dr67n-0 iAeYiQ Selectreact__SelectInput-sc-1shssly-0 cJLIjY"]')
        button.click()
    time.sleep(2)
    divelem = dr.find_elements(By.XPATH, value='.//li[@class="elements__StyledListItem-sc-197zmwo-0 QbTKh"]')
    divelem[6].click()
    time.sleep(3)
    
    try:
        alltimeavg = dr.find_element(By.XPATH, value='//div[@class="PriceHistoryStats--value"]').text
    except:
        print((collection[0], collection[1], floor, "no All Time Avg"))
        reslist.append([collection[0], collection[1], floor, "no All Time Avg"])
        continue
    print((collection[0], collection[1], floor, alltimeavg[1:]))
    reslist.append([collection[0], collection[1], floor, alltimeavg[1:]])
print(len(reslist))

with open('alltimeavg.csv', 'a', newline='', encoding='UTF-8') as f: 
    write = csv.writer(f) 
    write.writerows(reslist) 

(113, 'Duckclub', '2', '0.0035')
(114, 'SMATh World - Genesis Klaytn', '---', '2.4721')
(115, '4 Politics', '2.5', '0.0172')
(116, 'Klay Hodori', '10', '0.0162')
(117, 'TENZ Official', '56', '0.0804')
(118, 'KONGZ MINTER', '15', '0.008')
(119, 'Age of Zen Official', '350', '0.1618')
(120, 'Prime Baby Kongz', '11', '0.0263')
(121, 'Klaypick_soony', '50', '0.0097')
(122, 'Brave Slimes', '8', '0.0253')
(123, 'PLZ SWAP (NO MORE SUPPORTED)', '---', '0.0063')
(124, 'Klay Rudolph', '50', '0.0086')
(125, 'Angry Polar Bears', '2', '0.0242')
(126, 'KlayMonsterDressUp', '3.5', '0.0116')
(127, 'lonerrabbit', '3', '0.0019')
(128, 'PunkyHalloween', '3', '0.0026')
(129, 'SaveTheBlind_PFP', '45', '0.013')
(130, 'TREEMOJI', '5.6', '0.0098')
(131, 'TrendySeals', '1', '0.0088')
(132, 'KIaypad Key', '50', '0.08')
(133, 'Klay Hamster', '7', '0.0082')
(134, 'Kwang Boost', '69', '0.0803')
(135, 'KlayCats Student Union', '4.99', '0.0021')
(136, 'Toxic Ape Official', '40', '0.0501')
(137, 'MMT 500 Faces', '85'

IndexError: list index out of range

In [19]:
dr = wd.Chrome('chromedriver')
for collection in linklist[219:]:
    dr.get(collection[2]+"?tab=activity")
    time.sleep(4)
    infobox = dr.find_element(By.XPATH, value='//div[@class="Blockreact__Block-sc-1xf18x6-0 Flexreact__Flex-sc-1twd32i-0 InfoContainerreact__InfoContainer-sc-15x3z7c-0 CollectionStatsBarreact__Container-sc-8gdi9o-0 elqhCm jYqxGr fprnFG cWlCaZ"]')
    floorbox = infobox.find_elements(By.XPATH, value='*')[2]
    floor = floorbox.find_element(By.XPATH, value='.//div[@class="Overflowreact__OverflowContainer-sc-7qr9y8-0 jPSCbX"]').text

    try:
        button = dr.find_element(By.XPATH, value = '//div[@class="Inputreact__StyledContainer-sc-3dr67n-0 iAeYiQ Selectreact__SelectInput-sc-1shssly-0 cJLIjY"]')
    except:
        print((collection[0], collection[1], floor, "no activity"))
        reslist.append([collection[0], collection[1], floor, "no activity"])
        continue
    try:
        button.click()
    except:
        dr.get(collection[2]+"?tab=activity")
        time.sleep(4)
        infobox = dr.find_element(By.XPATH, value='//div[@class="Blockreact__Block-sc-1xf18x6-0 Flexreact__Flex-sc-1twd32i-0 InfoContainerreact__InfoContainer-sc-15x3z7c-0 CollectionStatsBarreact__Container-sc-8gdi9o-0 elqhCm jYqxGr fprnFG cWlCaZ"]')
        floorbox = infobox.find_elements(By.XPATH, value='*')[2]
        floor = floorbox.find_element(By.XPATH, value='.//div[@class="Overflowreact__OverflowContainer-sc-7qr9y8-0 jPSCbX"]').text
        button = dr.find_element(By.XPATH, value = '//div[@class="Inputreact__StyledContainer-sc-3dr67n-0 iAeYiQ Selectreact__SelectInput-sc-1shssly-0 cJLIjY"]')
        button.click()
    time.sleep(2)
    divelem = dr.find_elements(By.XPATH, value='.//li[@class="elements__StyledListItem-sc-197zmwo-0 QbTKh"]')
    divelem[6].click()
    time.sleep(3)
    
    try:
        alltimeavg = dr.find_element(By.XPATH, value='//div[@class="PriceHistoryStats--value"]').text
    except:
        print((collection[0], collection[1], floor, "no All Time Avg"))
        reslist.append([collection[0], collection[1], floor, "no All Time Avg"])
        continue
    print((collection[0], collection[1], floor, alltimeavg[1:]))
    reslist.append([collection[0], collection[1], floor, alltimeavg[1:]])
print(len(reslist))

with open('alltimeavg.csv', 'a', newline='', encoding='UTF-8') as f: 
    write = csv.writer(f) 
    write.writerows(reslist) 

(220, 'Marine Klay', '10', '0.0064')
(221, 'Dalgona Game', '2', '0.0019')
(222, 'Non Fungible Whale Coin', '65', '0.0013')
(223, 'KlayCute', '19', '0.01')
(224, 'Age of Zen Official', '---', '0.1')
(225, 'ApesPunks', '4', '0.0042')
(226, 'Greedy Bears', '18', '0.0059')
(227, 'Klaypad Avatar', '180', '0.1864')
(228, 'DOONG', '16', '0.0091')
(229, 'KlayOtter', '2', '0.0037')
(230, 'Hi_Bear', '9', '0.006')


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class="Blockreact__Block-sc-1xf18x6-0 Flexreact__Flex-sc-1twd32i-0 InfoContainerreact__InfoContainer-sc-15x3z7c-0 CollectionStatsBarreact__Container-sc-8gdi9o-0 elqhCm jYqxGr fprnFG cWlCaZ"]"}
  (Session info: chrome=99.0.4844.82)
Stacktrace:
Backtrace:
	Ordinal0 [0x00C79943+2595139]
	Ordinal0 [0x00C0C9F1+2148849]
	Ordinal0 [0x00B04528+1066280]
	Ordinal0 [0x00B30FD4+1249236]
	Ordinal0 [0x00B311CB+1249739]
	Ordinal0 [0x00B5D812+1431570]
	Ordinal0 [0x00B4BA34+1358388]
	Ordinal0 [0x00B5BAF2+1424114]
	Ordinal0 [0x00B4B806+1357830]
	Ordinal0 [0x00B26086+1204358]
	Ordinal0 [0x00B26F96+1208214]
	GetHandleVerifier [0x00E1B232+1658114]
	GetHandleVerifier [0x00ED312C+2411516]
	GetHandleVerifier [0x00D0F261+560433]
	GetHandleVerifier [0x00D0E366+556598]
	Ordinal0 [0x00C1286B+2173035]
	Ordinal0 [0x00C175F8+2192888]
	Ordinal0 [0x00C176E5+2193125]
	Ordinal0 [0x00C211FC+2232828]
	BaseThrea

In [20]:
dr = wd.Chrome('chromedriver')
for collection in linklist[230:]:
    dr.get(collection[2]+"?tab=activity")
    time.sleep(4)
    infobox = dr.find_element(By.XPATH, value='//div[@class="Blockreact__Block-sc-1xf18x6-0 Flexreact__Flex-sc-1twd32i-0 InfoContainerreact__InfoContainer-sc-15x3z7c-0 CollectionStatsBarreact__Container-sc-8gdi9o-0 elqhCm jYqxGr fprnFG cWlCaZ"]')
    floorbox = infobox.find_elements(By.XPATH, value='*')[2]
    floor = floorbox.find_element(By.XPATH, value='.//div[@class="Overflowreact__OverflowContainer-sc-7qr9y8-0 jPSCbX"]').text

    try:
        button = dr.find_element(By.XPATH, value = '//div[@class="Inputreact__StyledContainer-sc-3dr67n-0 iAeYiQ Selectreact__SelectInput-sc-1shssly-0 cJLIjY"]')
    except:
        print((collection[0], collection[1], floor, "no activity"))
        reslist.append([collection[0], collection[1], floor, "no activity"])
        continue
    try:
        button.click()
    except:
        dr.get(collection[2]+"?tab=activity")
        time.sleep(4)
        infobox = dr.find_element(By.XPATH, value='//div[@class="Blockreact__Block-sc-1xf18x6-0 Flexreact__Flex-sc-1twd32i-0 InfoContainerreact__InfoContainer-sc-15x3z7c-0 CollectionStatsBarreact__Container-sc-8gdi9o-0 elqhCm jYqxGr fprnFG cWlCaZ"]')
        floorbox = infobox.find_elements(By.XPATH, value='*')[2]
        floor = floorbox.find_element(By.XPATH, value='.//div[@class="Overflowreact__OverflowContainer-sc-7qr9y8-0 jPSCbX"]').text
        button = dr.find_element(By.XPATH, value = '//div[@class="Inputreact__StyledContainer-sc-3dr67n-0 iAeYiQ Selectreact__SelectInput-sc-1shssly-0 cJLIjY"]')
        button.click()
    time.sleep(2)
    divelem = dr.find_elements(By.XPATH, value='.//li[@class="elements__StyledListItem-sc-197zmwo-0 QbTKh"]')
    divelem[6].click()
    time.sleep(3)
    
    try:
        alltimeavg = dr.find_element(By.XPATH, value='//div[@class="PriceHistoryStats--value"]').text
    except:
        print((collection[0], collection[1], floor, "no All Time Avg"))
        reslist.append([collection[0], collection[1], floor, "no All Time Avg"])
        continue
    print((collection[0], collection[1], floor, alltimeavg[1:]))
    reslist.append([collection[0], collection[1], floor, alltimeavg[1:]])
print(len(reslist))

with open('alltimeavg.csv', 'a', newline='', encoding='UTF-8') as f: 
    write = csv.writer(f) 
    write.writerows(reslist) 

(231, 'Drunken Doll V0.1', '70', '0.0286')
(232, 'Ceramic Gardener', '20', '0.0101')
(233, 'GRD KRICE', '5', '0.0045')
(234, 'Age of Zen Heroes', '300', '0.1522')
(235, 'CYE ChuChuCAT V0 Official', '280', '0.0759')
(236, 'ChickiFarm Ornaments', '11', '0.0092')
(237, 'Duck in Donuts', '8.9', '0.0056')
(238, 'Martian Punks NFT', '---', '0.0057')
(239, 'cyber DOLLZ', '50', '0.0282')
(240, 'KAC Supporters', '---', '0.0228')
(241, 'Klay FoxFam', '20', '0.0048')
(242, 'llamacrypto', '15', '0.0024')
(243, 'KLAYMEKA', '5', '0.0051')
(244, '[Artists N X Soul Kim] The SoulCat 1256', '15', '0.0026')
(245, 'Crusera', '30', '0.0115')
(246, 'Little Ghost Town', '29', '0.0055')
(247, 'LGC LazyPeople Items', '11', '0.0059')
(248, 'META GHOST LAND V1', '5', '0.0062')
(249, 'Fluffy Squishy Mates', '---', '0.0027')
(250, 'Naguls Club', '7', '0.0042')
(251, 'Symbol Share', '---', '0.05')
(252, 'Crazy Crypto Sheep V2', '---', '0.0041')
(253, 'duckbean', '40', '0.0081')
(254, 'Crazy Stone Papa Club', '---',

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class="Blockreact__Block-sc-1xf18x6-0 Flexreact__Flex-sc-1twd32i-0 InfoContainerreact__InfoContainer-sc-15x3z7c-0 CollectionStatsBarreact__Container-sc-8gdi9o-0 elqhCm jYqxGr fprnFG cWlCaZ"]"}
  (Session info: chrome=99.0.4844.82)
Stacktrace:
Backtrace:
	Ordinal0 [0x00C79943+2595139]
	Ordinal0 [0x00C0C9F1+2148849]
	Ordinal0 [0x00B04528+1066280]
	Ordinal0 [0x00B30FD4+1249236]
	Ordinal0 [0x00B311CB+1249739]
	Ordinal0 [0x00B5D812+1431570]
	Ordinal0 [0x00B4BA34+1358388]
	Ordinal0 [0x00B5BAF2+1424114]
	Ordinal0 [0x00B4B806+1357830]
	Ordinal0 [0x00B26086+1204358]
	Ordinal0 [0x00B26F96+1208214]
	GetHandleVerifier [0x00E1B232+1658114]
	GetHandleVerifier [0x00ED312C+2411516]
	GetHandleVerifier [0x00D0F261+560433]
	GetHandleVerifier [0x00D0E366+556598]
	Ordinal0 [0x00C1286B+2173035]
	Ordinal0 [0x00C175F8+2192888]
	Ordinal0 [0x00C176E5+2193125]
	Ordinal0 [0x00C211FC+2232828]
	BaseThrea

In [21]:
dr = wd.Chrome('chromedriver')
for collection in linklist[270:]:
    dr.get(collection[2]+"?tab=activity")
    time.sleep(4)
    infobox = dr.find_element(By.XPATH, value='//div[@class="Blockreact__Block-sc-1xf18x6-0 Flexreact__Flex-sc-1twd32i-0 InfoContainerreact__InfoContainer-sc-15x3z7c-0 CollectionStatsBarreact__Container-sc-8gdi9o-0 elqhCm jYqxGr fprnFG cWlCaZ"]')
    floorbox = infobox.find_elements(By.XPATH, value='*')[2]
    floor = floorbox.find_element(By.XPATH, value='.//div[@class="Overflowreact__OverflowContainer-sc-7qr9y8-0 jPSCbX"]').text

    try:
        button = dr.find_element(By.XPATH, value = '//div[@class="Inputreact__StyledContainer-sc-3dr67n-0 iAeYiQ Selectreact__SelectInput-sc-1shssly-0 cJLIjY"]')
    except:
        print((collection[0], collection[1], floor, "no activity"))
        reslist.append([collection[0], collection[1], floor, "no activity"])
        continue
    try:
        button.click()
    except:
        dr.get(collection[2]+"?tab=activity")
        time.sleep(4)
        infobox = dr.find_element(By.XPATH, value='//div[@class="Blockreact__Block-sc-1xf18x6-0 Flexreact__Flex-sc-1twd32i-0 InfoContainerreact__InfoContainer-sc-15x3z7c-0 CollectionStatsBarreact__Container-sc-8gdi9o-0 elqhCm jYqxGr fprnFG cWlCaZ"]')
        floorbox = infobox.find_elements(By.XPATH, value='*')[2]
        floor = floorbox.find_element(By.XPATH, value='.//div[@class="Overflowreact__OverflowContainer-sc-7qr9y8-0 jPSCbX"]').text
        button = dr.find_element(By.XPATH, value = '//div[@class="Inputreact__StyledContainer-sc-3dr67n-0 iAeYiQ Selectreact__SelectInput-sc-1shssly-0 cJLIjY"]')
        button.click()
    time.sleep(2)
    divelem = dr.find_elements(By.XPATH, value='.//li[@class="elements__StyledListItem-sc-197zmwo-0 QbTKh"]')
    divelem[6].click()
    time.sleep(3)
    
    try:
        alltimeavg = dr.find_element(By.XPATH, value='//div[@class="PriceHistoryStats--value"]').text
    except:
        print((collection[0], collection[1], floor, "no All Time Avg"))
        reslist.append([collection[0], collection[1], floor, "no All Time Avg"])
        continue
    print((collection[0], collection[1], floor, alltimeavg[1:]))
    reslist.append([collection[0], collection[1], floor, alltimeavg[1:]])
print(len(reslist))

with open('alltimeavg.csv', 'a', newline='', encoding='UTF-8') as f: 
    write = csv.writer(f) 
    write.writerows(reslist) 

(271, 'animal low poly V2', '9', '0.0041')
(272, 'DieNo', '5', '0.0063')
(273, 'Ripho', '15', '0.0056')
(274, 'boxfrog', '---', '0.0184')
(275, 'Heroes of the Klay Official', '15', '0.0128')
(276, 'RICHMEME', '3', '0.0018')
(277, 'KlaySnowman', '7', '0.0026')
(278, 'treasure box_DEJAWA', '10.69', '0.0045')
(279, 'be_living', '---', '0.0041')
(280, 'Unknown_People', '10', '0.012')
(281, 'ChickiFarm Signs', '35', '0.0156')
(282, 'Klay Doodle Flowers', '0.8', '0.0027')
(283, 'Klay Zodiac EN', '---', '0.0211')
(284, 'DrinkableSnail', '---', '0.0026')
(285, 'DemonsPunks', '10', '0.0033')
(286, 'yoyqueen_k', '---', '0.0127')
(287, 'CHEWCHEWS klay', '---', '0.0076')
(288, 'WHAT WE ARE_WWA', '15', '0.0062')
(289, 'My Bunny World', '15', '0.0061')
(290, 'Planets of the Klay Galaxy', '5', '0.0026')
(291, 'BitcoinWolf', '1', '0.0111')
(292, 'HelloYellow', '30', '0.0111')
(293, 'Ceramic Gardener V2', '---', '0.002')
(294, 'The Klaytn Meta BOX', '0.25', '0.0045')
(295, 'Copy Bear', '2', '0.0062')
(

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class="Blockreact__Block-sc-1xf18x6-0 Flexreact__Flex-sc-1twd32i-0 InfoContainerreact__InfoContainer-sc-15x3z7c-0 CollectionStatsBarreact__Container-sc-8gdi9o-0 elqhCm jYqxGr fprnFG cWlCaZ"]"}
  (Session info: chrome=99.0.4844.82)
Stacktrace:
Backtrace:
	Ordinal0 [0x00C79943+2595139]
	Ordinal0 [0x00C0C9F1+2148849]
	Ordinal0 [0x00B04528+1066280]
	Ordinal0 [0x00B30FD4+1249236]
	Ordinal0 [0x00B311CB+1249739]
	Ordinal0 [0x00B5D812+1431570]
	Ordinal0 [0x00B4BA34+1358388]
	Ordinal0 [0x00B5BAF2+1424114]
	Ordinal0 [0x00B4B806+1357830]
	Ordinal0 [0x00B26086+1204358]
	Ordinal0 [0x00B26F96+1208214]
	GetHandleVerifier [0x00E1B232+1658114]
	GetHandleVerifier [0x00ED312C+2411516]
	GetHandleVerifier [0x00D0F261+560433]
	GetHandleVerifier [0x00D0E366+556598]
	Ordinal0 [0x00C1286B+2173035]
	Ordinal0 [0x00C175F8+2192888]
	Ordinal0 [0x00C176E5+2193125]
	Ordinal0 [0x00C211FC+2232828]
	BaseThrea

In [22]:
dr = wd.Chrome('chromedriver')
for collection in linklist[311:]:
    dr.get(collection[2]+"?tab=activity")
    time.sleep(4)
    infobox = dr.find_element(By.XPATH, value='//div[@class="Blockreact__Block-sc-1xf18x6-0 Flexreact__Flex-sc-1twd32i-0 InfoContainerreact__InfoContainer-sc-15x3z7c-0 CollectionStatsBarreact__Container-sc-8gdi9o-0 elqhCm jYqxGr fprnFG cWlCaZ"]')
    floorbox = infobox.find_elements(By.XPATH, value='*')[2]
    floor = floorbox.find_element(By.XPATH, value='.//div[@class="Overflowreact__OverflowContainer-sc-7qr9y8-0 jPSCbX"]').text

    try:
        button = dr.find_element(By.XPATH, value = '//div[@class="Inputreact__StyledContainer-sc-3dr67n-0 iAeYiQ Selectreact__SelectInput-sc-1shssly-0 cJLIjY"]')
    except:
        print((collection[0], collection[1], floor, "no activity"))
        reslist.append([collection[0], collection[1], floor, "no activity"])
        continue
    try:
        button.click()
    except:
        dr.get(collection[2]+"?tab=activity")
        time.sleep(4)
        infobox = dr.find_element(By.XPATH, value='//div[@class="Blockreact__Block-sc-1xf18x6-0 Flexreact__Flex-sc-1twd32i-0 InfoContainerreact__InfoContainer-sc-15x3z7c-0 CollectionStatsBarreact__Container-sc-8gdi9o-0 elqhCm jYqxGr fprnFG cWlCaZ"]')
        floorbox = infobox.find_elements(By.XPATH, value='*')[2]
        floor = floorbox.find_element(By.XPATH, value='.//div[@class="Overflowreact__OverflowContainer-sc-7qr9y8-0 jPSCbX"]').text
        button = dr.find_element(By.XPATH, value = '//div[@class="Inputreact__StyledContainer-sc-3dr67n-0 iAeYiQ Selectreact__SelectInput-sc-1shssly-0 cJLIjY"]')
        button.click()
    time.sleep(2)
    divelem = dr.find_elements(By.XPATH, value='.//li[@class="elements__StyledListItem-sc-197zmwo-0 QbTKh"]')
    divelem[6].click()
    time.sleep(3)
    
    try:
        alltimeavg = dr.find_element(By.XPATH, value='//div[@class="PriceHistoryStats--value"]').text
    except:
        print((collection[0], collection[1], floor, "no All Time Avg"))
        reslist.append([collection[0], collection[1], floor, "no All Time Avg"])
        continue
    print((collection[0], collection[1], floor, alltimeavg[1:]))
    reslist.append([collection[0], collection[1], floor, alltimeavg[1:]])
print(len(reslist))

with open('alltimeavg.csv', 'a', newline='', encoding='UTF-8') as f: 
    write = csv.writer(f) 
    write.writerows(reslist) 

(312, 'ParkKONGsoonKlay', '2', '0.0031')
(313, 'Glassy Dice', '---', '0.0019')
(314, 'Klay Owls', '5', '0.0027')
(315, 'Premium cat girl', '---', '0.0011')
(316, 'Lilis pixel art gallery', '25', '0.0067')
(317, 'Cutemong', '10', '0.005')
(318, 'Punk_Hands', '7', '0.0032')
(319, 'AWESOME PIXEL EARTH', '25', '0.0093')
(320, 'StarTales NFT', '---', '0.019')
(321, 'Riotn', '3', '0.0012')
(322, 'Klayperson', '---', '0.0029')
(323, 'Salamander is hungry', '9', '0.0042')
(324, 'GgumiGirl_Ver2_NFT', '---', '0.0082')
(325, 'HuhKyungYoungNFTs V2', '1.3', '0.0012')
(326, 'brag test', '---', '0.0031')
(327, 'Just Raccoonz :)', '15', '0.0051')
(328, 'Klay Noundles', '5', '0.0015')
(329, 'THE GUYS V2', '---', '0.0037')
(330, 'Jump Donet', '1', '0.00001')
(331, 'Dumbo Octopus', '13.2', '0.0042')
(332, 'KlayTopus', '---', '0.0027')
(333, 'levelzero-noname', '1', '0.0004')
(334, 'monsters-in-a-dream-by-jh', '30', '0.0112')
(335, 'JokoChan', '9.8', '0.0128')
(336, 'Klay MetaTravelers: Genesis Stone', '1

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class="Blockreact__Block-sc-1xf18x6-0 Flexreact__Flex-sc-1twd32i-0 InfoContainerreact__InfoContainer-sc-15x3z7c-0 CollectionStatsBarreact__Container-sc-8gdi9o-0 elqhCm jYqxGr fprnFG cWlCaZ"]"}
  (Session info: chrome=99.0.4844.82)
Stacktrace:
Backtrace:
	Ordinal0 [0x00C79943+2595139]
	Ordinal0 [0x00C0C9F1+2148849]
	Ordinal0 [0x00B04528+1066280]
	Ordinal0 [0x00B30FD4+1249236]
	Ordinal0 [0x00B311CB+1249739]
	Ordinal0 [0x00B5D812+1431570]
	Ordinal0 [0x00B4BA34+1358388]
	Ordinal0 [0x00B5BAF2+1424114]
	Ordinal0 [0x00B4B806+1357830]
	Ordinal0 [0x00B26086+1204358]
	Ordinal0 [0x00B26F96+1208214]
	GetHandleVerifier [0x00E1B232+1658114]
	GetHandleVerifier [0x00ED312C+2411516]
	GetHandleVerifier [0x00D0F261+560433]
	GetHandleVerifier [0x00D0E366+556598]
	Ordinal0 [0x00C1286B+2173035]
	Ordinal0 [0x00C175F8+2192888]
	Ordinal0 [0x00C176E5+2193125]
	Ordinal0 [0x00C211FC+2232828]
	BaseThrea

In [23]:
dr = wd.Chrome('chromedriver')
for collection in linklist[342:]:
    dr.get(collection[2]+"?tab=activity")
    time.sleep(4)
    infobox = dr.find_element(By.XPATH, value='//div[@class="Blockreact__Block-sc-1xf18x6-0 Flexreact__Flex-sc-1twd32i-0 InfoContainerreact__InfoContainer-sc-15x3z7c-0 CollectionStatsBarreact__Container-sc-8gdi9o-0 elqhCm jYqxGr fprnFG cWlCaZ"]')
    floorbox = infobox.find_elements(By.XPATH, value='*')[2]
    floor = floorbox.find_element(By.XPATH, value='.//div[@class="Overflowreact__OverflowContainer-sc-7qr9y8-0 jPSCbX"]').text

    try:
        button = dr.find_element(By.XPATH, value = '//div[@class="Inputreact__StyledContainer-sc-3dr67n-0 iAeYiQ Selectreact__SelectInput-sc-1shssly-0 cJLIjY"]')
    except:
        print((collection[0], collection[1], floor, "no activity"))
        reslist.append([collection[0], collection[1], floor, "no activity"])
        continue
    try:
        button.click()
    except:
        dr.get(collection[2]+"?tab=activity")
        time.sleep(4)
        infobox = dr.find_element(By.XPATH, value='//div[@class="Blockreact__Block-sc-1xf18x6-0 Flexreact__Flex-sc-1twd32i-0 InfoContainerreact__InfoContainer-sc-15x3z7c-0 CollectionStatsBarreact__Container-sc-8gdi9o-0 elqhCm jYqxGr fprnFG cWlCaZ"]')
        floorbox = infobox.find_elements(By.XPATH, value='*')[2]
        floor = floorbox.find_element(By.XPATH, value='.//div[@class="Overflowreact__OverflowContainer-sc-7qr9y8-0 jPSCbX"]').text
        button = dr.find_element(By.XPATH, value = '//div[@class="Inputreact__StyledContainer-sc-3dr67n-0 iAeYiQ Selectreact__SelectInput-sc-1shssly-0 cJLIjY"]')
        button.click()
    time.sleep(2)
    divelem = dr.find_elements(By.XPATH, value='.//li[@class="elements__StyledListItem-sc-197zmwo-0 QbTKh"]')
    divelem[6].click()
    time.sleep(3)
    
    try:
        alltimeavg = dr.find_element(By.XPATH, value='//div[@class="PriceHistoryStats--value"]').text
    except:
        print((collection[0], collection[1], floor, "no All Time Avg"))
        reslist.append([collection[0], collection[1], floor, "no All Time Avg"])
        continue
    print((collection[0], collection[1], floor, alltimeavg[1:]))
    reslist.append([collection[0], collection[1], floor, alltimeavg[1:]])
print(len(reslist))

with open('alltimeavg.csv', 'a', newline='', encoding='UTF-8') as f: 
    write = csv.writer(f) 
    write.writerows(reslist) 

(343, 'PEANUT FAIRY', '10', '0.0042')
(344, 'Cartoons in town', '---', '0.002')
(345, 'KlayKitties', '9.9', '0.0075')
(346, 'SYLTARE-OFFIClAL', '190', '0.0035')
(347, 'seKret Kosmic Kats', '0.002', '0.0001')
(348, 'Killer Girlfriend #KGF', '4', '0.0021')
(349, '55FRIENDS', '7', '0.0028')
(350, 'NFTournaments', '10', '0.0002')
(351, 'Azuki Ape', '10', '0.0042')
(352, 'DSC2 WELCOME EGGS FOR OUR NEW FRIENDS', '---', '0.001')
(353, 'LostDinoEgg2 Klay', '10', '0.004')
(354, 'AlienOnion', '5', '0.0041')
(355, 'Alpha Radar', '18', '0.0039')
(356, 'NEON YUZU', '0.25', '0.0001')
(357, 'chophi nft collection klay', '---', '0.0019')
(358, 'PPEEPP NFT', '10', '0.0037')


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class="Blockreact__Block-sc-1xf18x6-0 Flexreact__Flex-sc-1twd32i-0 InfoContainerreact__InfoContainer-sc-15x3z7c-0 CollectionStatsBarreact__Container-sc-8gdi9o-0 elqhCm jYqxGr fprnFG cWlCaZ"]"}
  (Session info: chrome=99.0.4844.82)
Stacktrace:
Backtrace:
	Ordinal0 [0x00C79943+2595139]
	Ordinal0 [0x00C0C9F1+2148849]
	Ordinal0 [0x00B04528+1066280]
	Ordinal0 [0x00B30FD4+1249236]
	Ordinal0 [0x00B311CB+1249739]
	Ordinal0 [0x00B5D812+1431570]
	Ordinal0 [0x00B4BA34+1358388]
	Ordinal0 [0x00B5BAF2+1424114]
	Ordinal0 [0x00B4B806+1357830]
	Ordinal0 [0x00B26086+1204358]
	Ordinal0 [0x00B26F96+1208214]
	GetHandleVerifier [0x00E1B232+1658114]
	GetHandleVerifier [0x00ED312C+2411516]
	GetHandleVerifier [0x00D0F261+560433]
	GetHandleVerifier [0x00D0E366+556598]
	Ordinal0 [0x00C1286B+2173035]
	Ordinal0 [0x00C175F8+2192888]
	Ordinal0 [0x00C176E5+2193125]
	Ordinal0 [0x00C211FC+2232828]
	BaseThrea

In [ ]:
ㅣㅑㅜ